Copyright (c) Microsoft Corporation.  
Licensed under the MIT License.  
  

# FWI demo based on: 
This project ports devito (https://github.com/opesci/devito) into Azure and runs tutorial notebooks at:
https://nbviewer.jupyter.org/github/opesci/devito/blob/master/examples/seismic/tutorials/



In this notebook we run the devito demo [notebooks](https://nbviewer.jupyter.org/github/opesci/devito/blob/master/examples/seismic/tutorials/) mentioned above by using an [AzureML estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py) with custom docker image. The docker image and associated docker file were created in previous notebook.

<a id='devito_in_AzureML_demoing_modes'></a>
####   This notebook is used as a control plane to submit experimentation jobs running devito in Azure in two modes (see [remote run azureml python script file invoking devito](#devito_demo_mode)):
 - [Mode 1](#devito_demo_mode_1):
      - uses custom code (slightly modified graphing functions save images to files too) 
      - experimentation job is defined by the devito code that is packaged as a py file to be run on an Azure remote compute target
      - experimentation job can be used to track metrics or other artifacts (images)
  
 - Mode 2:
      - papermill is invoked via cli or via its Python API to run unedited devito demo notebooks (https://github.com/opesci/devito/tree/master/examples/seismic/tutorials) on the remote compute target and get back the results as saved notebooks that are then Available in Azure portal. 


In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import sys, os
import shutil
import urllib
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import MpiConfiguration


# from azureml.core.datastore import Datastore
# from azureml.data.data_reference import DataReference
# from azureml.pipeline.steps import HyperDriveStep
# from azureml.pipeline.core import Pipeline, PipelineData
# from azureml.train.dnn import TensorFlow

from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

import platform

In [3]:
print("Azure ML SDK Version: ", azureml.core.VERSION)
platform.platform()
os.getcwd()

Azure ML SDK Version:  1.0.74


'Linux-4.15.0-1063-azure-x86_64-with-debian-10.1'

'/workspace/contrib/examples/imaging/azureml_devito/notebooks'

In [4]:
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
        
auxiliary_files_dir = os.path.join(*(['.', 'src']))
paths_to_append = [os.path.join(os.getcwd(), auxiliary_files_dir)]
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import project_utils
prj_consts = project_utils.project_consts()

dotenv_file_path = os.path.join(*(prj_consts.DOTENV_FILE_PATH))
dotenv_file_path

[None]

'./../not_shared/general.env'

In [5]:
%load_ext dotenv

In [6]:
workspace_config_dir = os.path.join(*(prj_consts.AML_WORKSPACE_CONFIG_DIR))
workspace_config_file = prj_consts.AML_WORKSPACE_CONFIG_FILE_NAME
workspace_config_dir

'./../not_shared'

In [7]:
%dotenv $dotenv_file_path

script_folder = prj_consts.AML_EXPERIMENT_DIR + ['devito_tutorial']

devito_training_script_file = '01_modelling.py' # hardcoded in file azureml_training_script_full_file_name below
azureml_training_script_file = 'azureml_'+devito_training_script_file
experimentName = '020_AzureMLEstimator'

os.makedirs(os.path.join(*(script_folder)), exist_ok=True)
script_path = os.path.join(*(script_folder))
training_script_full_file_name = os.path.join(script_path, devito_training_script_file)
azureml_training_script_full_file_name = os.path.join(script_path, azureml_training_script_file)

training_script_full_file_name
azureml_training_script_full_file_name

'./../temp/devito_tutorial/01_modelling.py'

'./../temp/devito_tutorial/azureml_01_modelling.py'

<a id='devito_demo_mode_1'></a>
 
##### devito in Azure ML demo mode 1
Create devito demo script based on 
https://nbviewer.jupyter.org/github/opesci/devito/blob/master/examples/seismic/tutorials/01_modelling.ipynb

[Back](#devito_in_AzureML_demoing_modes) to summary of modes od demoing devito in AzureML.

Main purpose of this script is to extend _plot_velocity()_ and _plot_shotrecord()_ devito [plotting functions](https://github.com/opesci/devito/blob/master/examples/seismic/plotting.py) to allow the mto work in batch mode, i.e. save output to a file.

In [8]:
%%writefile $training_script_full_file_name

import numpy as np
import os, argparse

from examples.seismic import Model
from examples.seismic import TimeAxis
from examples.seismic import Receiver
from devito import TimeFunction
from devito import Eq, solve
from devito import Operator


# try:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.rc('font', size=16)
mpl.rc('figure', figsize=(8, 6))
# except:
#     plt = None
#     cm = None
        


# "all" plotting utils in devito do not save to file, so we extend them here
# https://github.com/opesci/devito/blob/master/examples/seismic/plotting.py
def plot_velocity(model, source=None, receiver=None, colorbar=True, file=None):
    """
    Plot a two-dimensional velocity field from a seismic `Model`
    object. Optionally also includes point markers for sources and receivers.

    Parameters
    ----------
    model : Model
        Object that holds the velocity model.
    source : array_like or float
        Coordinates of the source point.
    receiver : array_like or float
        Coordinates of the receiver points.
    colorbar : bool
        Option to plot the colorbar.
    """
    domain_size = 1.e-3 * np.array(model.domain_size)
    extent = [model.origin[0], model.origin[0] + domain_size[0],
              model.origin[1] + domain_size[1], model.origin[1]]

    plot = plt.imshow(np.transpose(model.vp.data), animated=True, cmap=cm.jet,
                      vmin=np.min(model.vp.data), vmax=np.max(model.vp.data),
                      extent=extent)
    plt.xlabel('X position (km)')
    plt.ylabel('Depth (km)')

    # Plot source points, if provided
    if receiver is not None:
        plt.scatter(1e-3*receiver[:, 0], 1e-3*receiver[:, 1],
                    s=25, c='green', marker='D')

    # Plot receiver points, if provided
    if source is not None:
        plt.scatter(1e-3*source[:, 0], 1e-3*source[:, 1],
                    s=25, c='red', marker='o')

    # Ensure axis limits
    plt.xlim(model.origin[0], model.origin[0] + domain_size[0])
    plt.ylim(model.origin[1] + domain_size[1], model.origin[1])

    # Create aligned colorbar on the right
    if colorbar:
        ax = plt.gca()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        cbar = plt.colorbar(plot, cax=cax)
        cbar.set_label('Velocity (km/s)')
    plt.show()
    
    if file is not None:
        plt.savefig(file)
        print('plotted image saved as {} file'.format(file))
        
    plt.clf()

def plot_shotrecord(rec, model, t0, tn, colorbar=True, file=None):
    """
    Plot a shot record (receiver values over time).

    Parameters
    ----------
    rec :
        Receiver data with shape (time, points).
    model : Model
        object that holds the velocity model.
    t0 : int
        Start of time dimension to plot.
    tn : int
        End of time dimension to plot.
    """
    scale = np.max(rec) / 10.
    extent = [model.origin[0], model.origin[0] + 1e-3*model.domain_size[0],
              1e-3*tn, t0]

    plot = plt.imshow(rec, vmin=-scale, vmax=scale, cmap=cm.gray, extent=extent)
    plt.xlabel('X position (km)')
    plt.ylabel('Time (s)')

    # Create aligned colorbar on the right
    if colorbar:
        ax = plt.gca()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(plot, cax=cax)
    plt.show()   
    
    if file is not None:
        plt.savefig(file)
        print('plotted image saved as {} file'.format(file))
        
    plt.clf()

def main(output_folder):      
    # 1. Define the physical problem
    # The first step is to define the physical model:
    #  - physical dimensions of interest
    #  - velocity profile of this physical domain

    # Define a physical size
    shape = (101, 101)  # Number of grid point (nx, nz)
    spacing = (10., 10.)  # Grid spacing in m. The domain size is now 1km by 1km
    origin = (0., 0.)  # What is the location of the top left corner. This is necessary to define
    # the absolute location of the source and receivers

    # Define a velocity profile. The velocity is in km/s
    v = np.empty(shape, dtype=np.float32)
    v[:, :51] = 1.5
    v[:, 51:] = 2.5

    # With the velocity and model size defined, we can create the seismic model that
    # encapsulates this properties. We also define the size of the absorbing layer as 10 grid points
    model = Model(vp=v, origin=origin, shape=shape, spacing=spacing,
                  space_order=2, nbpml=10)

    plot_velocity(model, 
              file= os.path.join(*( [output_folder,'output000.png'])))
    
    # 2. Acquisition geometry
    t0 = 0.  # Simulation starts a t=0
    tn = 1000.  # Simulation last 1 second (1000 ms)
    dt = model.critical_dt  # Time step from model grid spacing

    time_range = TimeAxis(start=t0, stop=tn, step=dt)
    from examples.seismic import RickerSource

    f0 = 0.010  # Source peak frequency is 10Hz (0.010 kHz)
    src = RickerSource(name='src', grid=model.grid, f0=f0,
                       npoint=1, time_range=time_range)

    # First, position source centrally in all dimensions, then set depth
    src.coordinates.data[0, :] = np.array(model.domain_size) * .5
    src.coordinates.data[0, -1] = 20.  # Depth is 20m

    # We can plot the time signature to see the wavelet
#     src.show()

    # Create symbol for 101 receivers
    rec = Receiver(name='rec', grid=model.grid, npoint=101, time_range=time_range)

    # Prescribe even spacing for receivers along the x-axis
    rec.coordinates.data[:, 0] = np.linspace(0, model.domain_size[0], num=101)
    rec.coordinates.data[:, 1] = 20.  # Depth is 20m

    # We can now show the source and receivers within our domain:
    # Red dot: Source location
    # Green dots: Receiver locations (every 4th point)
    plot_velocity(model, source=src.coordinates.data,
                  receiver=rec.coordinates.data[::4, :], 
              file= os.path.join(*( [output_folder,'output010.png'])))
    
    # Define the wavefield with the size of the model and the time dimension
    u = TimeFunction(name="u", grid=model.grid, time_order=2, space_order=2)

    # We can now write the PDE
    pde = model.m * u.dt2 - u.laplace + model.damp * u.dt

    # The PDE representation is as on paper
    pde
    
    # This discrete PDE can be solved in a time-marching way updating u(t+dt) from the previous time step
    # Devito as a shortcut for u(t+dt) which is u.forward. We can then rewrite the PDE as 
    # a time marching updating equation known as a stencil using customized SymPy functions

    stencil = Eq(u.forward, solve(pde, u.forward))
    # Finally we define the source injection and receiver read function to generate the corresponding code
    src_term = src.inject(field=u.forward, expr=src * dt**2 / model.m)

    # Create interpolation expression for receivers
    rec_term = rec.interpolate(expr=u.forward)

    op = Operator([stencil] + src_term + rec_term, subs=model.spacing_map)
    
    op(time=time_range.num-1, dt=model.critical_dt)
    plot_shotrecord(rec.data, model, t0, tn, 
              file= os.path.join(*( [output_folder,'output020.png'])))

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--output_folder', type=str, nargs='?', \
                        dest='output_folder', help='ouput artifacts location',\
                       default='.')
    args = parser.parse_args()
    
    main(args.output_folder)

Overwriting ./../temp/devito_tutorial/01_modelling.py


##### Get experimentation docker image for devito

In [9]:
docker_repo_name = os.getenv('ACR_NAME')+'.azurecr.io' # or os.getenv('DOCKER_LOGIN')

docker_image_name = os.getenv('EXPERIMENTATION_DOCKER_IMAGE_NAME')

image_version = os.getenv('EXPERIMENTATION_DOCKER_IMAGE_TAG')
if image_version!="":
    docker_image_name = docker_image_name +':'+ image_version

full_docker_image_name = docker_repo_name + '/' + docker_image_name
    
docker_image_name
full_docker_image_name

'fwi01_azureml:sdk.v1.0.74'

'fwi01acr.azurecr.io/fwi01_azureml:sdk.v1.0.74'

Extract/decide the python path in custom docker image that corresponds to desired conda environment. Without this, AzureML tries to create a separate environment.

In [10]:
get_Python_path_command='docker run -i --rm  --name fwi01_azureml_container02 '+ \
full_docker_image_name + \
' /bin/bash -c "which python" '
get_Python_path_command


import subprocess
python_path_in_docker_image = subprocess.check_output(get_Python_path_command,shell=True,stderr=subprocess.STDOUT).\
decode('utf-8').strip()
python_path_in_docker_image

'docker run -i --rm  --name fwi01_azureml_container02 fwi01acr.azurecr.io/fwi01_azureml:sdk.v1.0.74 /bin/bash -c "which python" '

'/opt/conda/envs/fwi01_conda_env/bin/python'

<a id='devito_demo_mode'></a>
#### Create azureml_script_file that invokes:
 - devito exclusive custom edited training_script_file
 - unedited devito notebooks via papermill (invoked via cli and via ppapermill python API)

[Back](#devito_in_AzureML_demoing_modes) to notebook summary.

In [11]:
%%writefile $azureml_training_script_full_file_name

import argparse
import os
os.system('conda env list')

import azureml.core;
from azureml.core.run import Run

print(azureml.core.VERSION)

parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', help='ouput artifacts location')

args = parser.parse_args()
print('args.output_folder is {} but it will be ignored since AzureML_tracked ./outputs will be used'.format(args.output_folder))

# get the Azure ML run object
run = Run.get_context()

# ./outputs/ folder is autotracked so should get uploaded at the end of the run
output_dir_AzureML_tracked = './outputs'

crt_dir = os.getcwd()

cli_command= \
'cd /devito; /opt/conda/envs/fwi01_conda_env/bin/python '+ crt_dir +'/01_modelling.py' + \
' --output_folder '+ crt_dir + output_dir_AzureML_tracked+ '/' + \
' > '+ crt_dir + output_dir_AzureML_tracked + '/01_modelling.log' 
# + \
# ' 2>&1 ' + crt_dir +'/'+ output_dir_AzureML_tracked + '/devito_cli_py.log'
print('Running devito from cli on 01_modelling.py----BEGIN-----:') 
print(cli_command); print('\n');os.system(cli_command)
print('Running devito from cli on 01_modelling.py----END-----:\n\n')

cli_command= \
'cd /devito; papermill ' + \
'./examples/seismic/tutorials/02_rtm.ipynb '+\
crt_dir +'/outputs/02_rtm_output.ipynb  ' + \
'--log-output  --no-progress-bar  --kernel python3 ' + \
' > '+ crt_dir + output_dir_AzureML_tracked + '/02_rtm_output.log' 
# + \
# ' 2>&1 ' + crt_dir +'/'+ output_dir_AzureML_tracked + '/papermill_cli.log'

# FIXME - activate right conda env for running papermill from cli
activate_right_conda_env_fixed = False
if activate_right_conda_env_fixed:
    print('Running papermill from cli on 02_rtm.ipynb----BEGIN-----:') 
    print(cli_command); print('\n');os.system(cli_command)
    print('Running papermill from cli on 02_rtm.ipynb----END-----:\n\n') 


print('Running papermill from Python API on 03_fwi.ipynb----BEGIN-----:') 
import papermill as pm
os.chdir('/devito')
pm.execute_notebook(
   './examples/seismic/tutorials/03_fwi.ipynb',
   crt_dir +'/outputs/03_fwi_output.ipynb'
)
print('Running papermill from Python API on 03_fwi.ipynb----END-----:') 

print('Running papermill from Python API on 04_dask.ipynb----BEGIN-----:') 
import papermill as pm
os.chdir('/devito')
pm.execute_notebook(
   './examples/seismic/tutorials/04_dask.ipynb',
   crt_dir +'/outputs/04_dask_output.ipynb'
)
print('Running papermill from Python API on 04_dask.ipynb----END-----:') 
 

os.system('pwd')
os.system('ls -l /')
os.system('ls -l ./')
os.system('ls -l ' +crt_dir + output_dir_AzureML_tracked)
run.log('training_message01: ', 'finished experiment')
print('\n')

Overwriting ./../temp/devito_tutorial/azureml_01_modelling.py


In [12]:
script_path=os.path.join(*(script_folder))
os.listdir(script_path)

['azureml_01_modelling.py', '01_modelling.py']

## Initialize workspace

Initialize a workspace object from persisted configuration. If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure the config file is present at .\config.json

In [13]:
ws = Workspace.from_config(
    path=os.path.join(os.getcwd(),
                      os.path.join(*([workspace_config_dir, '.azureml', workspace_config_file]))))
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id[0:4], sep = '\n')

Workspace name: ghiordanfwiws
Azure region: eastus2
Subscription id: 7899


## Create an Azure ML experiment
Let's create an experiment named "tf-mnist" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

In [14]:
exp = Experiment(workspace=ws, name=experimentName)

## Retrieve or create a Azure Machine Learning compute
Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Azure Machine Learning Compute in the current workspace, if it doesn't already exist. We will then run the training script on this compute target.

If we could not find the compute with the given name in the previous cell, then we will create a new compute here. This process is broken down into the following steps:

1. Create the configuration
2. Create the Azure Machine Learning compute

**This process will take a few minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.**

In [15]:
gpu_cluster_name = os.getenv('GPU_CLUSTER_NAME')
gpu_cluster_name = 'gpuclstfwi08'
gpu_cluster_name

'gpuclstfwi08'

In [16]:
# Verify that cluster does not exist already
max_nodes_value = 5
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Could not find ComputeTarget cluster!")
    
# #     Create a new gpucluster using code below
#     # Specify the configuration for the new cluster
#     compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_NC6",
#                                                            min_nodes=0,
#                                                            max_nodes=max_nodes_value)
#     # Create the cluster with the specified name and configuration
#     gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

#     # Wait for the cluster to complete, show the output log
#     gpu_cluster.wait_for_completion(show_output=True)
    
    
#     for demo purposes, show how clsuter properties can be altered post-creation
gpu_cluster.update(min_nodes=0, max_nodes=max_nodes_value, idle_seconds_before_scaledown=1200)

Found existing gpu cluster


#### Create an Azure ML SDK estimator with custom docker image 

In [17]:
# use a custom Docker image
from azureml.core.container_registry import ContainerRegistry

image_name = docker_image_name

# you can also point to an image in a private ACR
image_registry_details = ContainerRegistry()
image_registry_details.address = docker_repo_name
image_registry_details.username = os.getenv('ACR_USERNAME')
image_registry_details.password = os.getenv('ACR_PASSWORD') 

# don't let the system build a new conda environment
user_managed_dependencies = True

# submit to a local Docker container. if you don't have Docker engine running locally, you can set compute_target to cpu_cluster.
script_params = {
        '--output_folder': 'some_folder'
}


# distributed_training_conf = MpiConfiguration()
# distributed_training_conf.process_count_per_node = 2

est = Estimator(source_directory=script_path, 
                compute_target=gpu_cluster,#'local', #gpu_cluster, 
                entry_script=azureml_training_script_file,
                script_params=script_params,
                use_docker=True,
                custom_docker_image=image_name,
                # uncomment below line to use your private ACR
                image_registry_details=image_registry_details, 
                user_managed=user_managed_dependencies,
                distributed_training=None,
                node_count=1
                )
est.run_config.environment.python.interpreter_path = python_path_in_docker_image

run = exp.submit(est)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

One can use the above link to currrent experiment run in Azure Portal to see tracked metrics, and images and output notebooks saved by azureml_training_script_full_file_name in {run_dir}/outputs on the remote compute target that are automatically saved by AzureML in the run history Azure portal pages.

In [18]:
response = run.wait_for_completion(show_output=False)

In [19]:
import time
from collections import Counter
#wait till all jobs finished

def wait_for_run_list_to_finish(the_run_list):
    finished_status_list = ['Completed', 'Failed']
    printing_counter = 0
    start_time = time.time()
    while (not all((crt_queried_job.get_status() in finished_status_list) for crt_queried_job in the_run_list)):
        time.sleep(2)
        printing_counter+= 1
        print('print {0:.0f}, time {1:.3f} seconds: {2}'.format(printing_counter, time.time() - start_time, 
                                str(Counter([crt_queried_job.get_status() for crt_queried_job in the_run_list]))), end="\r")
#     final status
    print('Final print {0:.0f}, time {1:.3f} seconds: {2}'.format(printing_counter, time.time() - start_time, 
                                str(Counter([crt_queried_job.get_status() for crt_queried_job in the_run_list]))), end="\r")     
wait_for_run_list_to_finish([run])

In [20]:
import datetime, math
def get_run_duration(azureml_exp_run):
    run_details =  azureml_exp_run.get_details()
    run_duration = datetime.datetime.strptime(run_details['endTimeUtc'], "%Y-%m-%dT%H:%M:%S.%fZ") - \
               datetime.datetime.strptime(run_details['startTimeUtc'], "%Y-%m-%dT%H:%M:%S.%fZ")
    return run_duration.total_seconds()
run_duration = get_run_duration(run)

run_seconds, run_minutes = math.modf(run_duration/60)
print('run_duration in seconds {}'.format(run_duration))
print('run_duration= {0:.0f}m {1:.3f}s'.format(run_minutes, run_seconds*60))


run_duration in seconds 108.573284
run_duration= 1m 48.573s


In [21]:
import time
from IPython.display import clear_output

no_of_jobs = 20
no_of_nodes = 10

job_counter = 0
print_cycle = 7
run_list = []
submit_time_list = []
for crt_nodes in range(no_of_nodes, (no_of_nodes+1)):
    gpu_cluster.update(min_nodes=0, max_nodes=crt_nodes, idle_seconds_before_scaledown=1200)
    clust_start_time = time.time()
    for crt_job in range(1, no_of_jobs):
        job_counter+= 1
        start_time = time.time()
        run = exp.submit(est)
        end_time = time.time()
        run_time = end_time - start_time
        run_list.append(run)
        submit_time_list.append(run_time)
        print('Counter{}: submission of job {} on {} nodes took {} seconds '.format(job_counter, crt_job, crt_nodes, run_time))
        print('run list length {}'.format(len(run_list)))
        if ((job_counter-1) % print_cycle) == 0:
            clear_output()
            print('Showing details for run {}'.format(job_counter))
            RunDetails(run).show()
#     [all_jobs_done = True if (('Completed'==crt_queried_job.get_status()) for crt_queried_job in run_list)]

Showing details for run 15


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Counter16: submission of job 16 on 10 nodes took 8.287715435028076 seconds 
run list length 16
Counter17: submission of job 17 on 10 nodes took 7.936461687088013 seconds 
run list length 17
Counter18: submission of job 18 on 10 nodes took 8.747953414916992 seconds 
run list length 18
Counter19: submission of job 19 on 10 nodes took 7.859047889709473 seconds 
run list length 19


In [22]:
import numpy as np
np.asarray(submit_time_list)
np.histogram(np.asarray(submit_time_list), bins=np.linspace(6.0, 10.0, num=10), density=False)

array([ 9.95614195, 11.63660598, 11.59176159,  8.24005198, 39.15067554,
        7.82959986, 10.54416084,  7.90288782,  9.45449305,  8.52095628,
       12.79820418,  8.11050415, 19.89081311,  7.72854948,  8.98733926,
        8.28771544,  7.93646169,  8.74795341,  7.85904789])

(array([0, 0, 0, 1, 5, 3, 2, 1, 1]),
 array([ 6.        ,  6.44444444,  6.88888889,  7.33333333,  7.77777778,
         8.22222222,  8.66666667,  9.11111111,  9.55555556, 10.        ]))

In [23]:
def wait_for_run_list_to_finish(the_run_list, plot_results=True):
    finished_status_list = ['Completed', 'Failed']
    printing_counter = 0
    start_time = time.time()
    while (not all((crt_queried_job.get_status() in finished_status_list) for crt_queried_job in the_run_list)):
        time.sleep(2)
        printing_counter+= 1
        crt_status = Counter([crt_queried_job.get_status() for crt_queried_job in the_run_list])
        print('print {0:.0f}, time {1:.3f} seconds: {2}'.format(printing_counter, time.time() - start_time, 
                                str(crt_status)), end="\r")
        if plot_results:
#           import numpy as np
            import matplotlib.pyplot as plt
            plt.bar(crt_status.keys(), crt_status.values())
            plt.show()
            
#             indexes = np.arange(len(labels))
#             width = 1

#             plt.bar(indexes, values, width)
#             plt.xticks(indexes + width * 0.5, labels)
#             plt.show()

#             from pandas import Series
#             crt_status = Series([crt_queried_job.get_status() for crt_queried_job in the_run_list])
#             status_counts = crt_status.value_counts().sort_index()
#             print('print {0:.0f}, time {1:.3f} seconds: {2}'.format(printing_counter, time.time() - start_time, 
#                                 str(status_counts)), end="\r")
#     final status
    print('Final print {0:.0f}, time {1:.3f} seconds: {2}'.format(printing_counter, time.time() - start_time, 
                                str(Counter([crt_queried_job.get_status() for crt_queried_job in the_run_list]))), end="\r")     

    
    
wait_for_run_list_to_finish(run_list, plot_results=False)

In [24]:
run_durations = [get_run_duration(crt_queried_job) for crt_queried_job in run_list]
run_statuses = [crt_queried_job.get_status() for crt_queried_job in run_list]

In [25]:
run_durations = np.asarray(run_durations)
run_statuses = np.asarray(run_statuses)

extreme_k = 20
#longest runs
indices = np.argsort(run_durations)[-extreme_k:]
indices
print(run_durations[indices])
print(run_statuses[indices])
#shortest runs
indices = np.argsort(run_durations)[0:extreme_k]
indices
print(run_durations[indices])
print(run_statuses[indices])

#run_durations histogram - counts and bins
np.histogram(run_durations, bins=np.linspace(50, 200, num=10), density=False)


array([ 0,  9,  4, 13, 10,  5, 14, 18,  7,  8, 16, 11,  6,  1, 12, 15,  3,
        2, 17])

[  4.819322 101.946523 106.5569   107.866548 109.776871 111.461206
 113.732314 117.302445 122.746341 135.938763 156.116081 157.700891
 158.001392 162.059693 163.769164 164.909099 166.151158 175.955566
 185.333479]
['Failed' 'Completed' 'Completed' 'Completed' 'Completed' 'Completed'
 'Completed' 'Completed' 'Failed' 'Failed' 'Completed' 'Completed'
 'Completed' 'Completed' 'Completed' 'Completed' 'Failed' 'Failed'
 'Failed']


array([ 0,  9,  4, 13, 10,  5, 14, 18,  7,  8, 16, 11,  6,  1, 12, 15,  3,
        2, 17])

[  4.819322 101.946523 106.5569   107.866548 109.776871 111.461206
 113.732314 117.302445 122.746341 135.938763 156.116081 157.700891
 158.001392 162.059693 163.769164 164.909099 166.151158 175.955566
 185.333479]
['Failed' 'Completed' 'Completed' 'Completed' 'Completed' 'Completed'
 'Completed' 'Completed' 'Failed' 'Failed' 'Completed' 'Completed'
 'Completed' 'Completed' 'Completed' 'Completed' 'Failed' 'Failed'
 'Failed']


(array([0, 0, 0, 6, 2, 1, 7, 1, 1]),
 array([ 50.        ,  66.66666667,  83.33333333, 100.        ,
        116.66666667, 133.33333333, 150.        , 166.66666667,
        183.33333333, 200.        ]))

In [26]:
print('Finished running 030_ScaleJobsUsingAzuremL_GeophysicsTutorial_FWI_Azure_devito!')

Finished running 030_ScaleJobsUsingAzuremL_GeophysicsTutorial_FWI_Azure_devito!
